In [105]:
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, CustomJS, Slider
from bokeh.layouts import widgetbox, column
output_notebook()

Loading BokehJS ...

In [135]:
def telaraña(f, x, xiniciales, nmax=200):
    "Dibuja el diagrama de telaraña de f sobre el arreglo x, con valor(es) inicial(es) xinic."
    "grafica la función sobre x"
    p=figure()
    p.line(x, f(x), color='black', line_width=3)
    "diagonal"
    p.line(x, x, color="red", line_width=3)
    sliders = [None]
    def iteraciones(xinic):
        "iteraciones"
        x1 = np.empty(nmax//2)
        x1[0] = xinic
        _f = np.frompyfunc(lambda x,_: f(x), 2, 1) # accumulate requiere función binaria pero f se asume unaria
        x2 = _f.accumulate(x1, dtype=np.object).astype(np.float)
        "duplicando"
        x3 = np.repeat(x2, 2)
        "construyendo y"
        y1 = f(x3)
        y3 = np.concatenate(([0], y1[:-1]))
        source = ColumnDataSource(data=dict(x=x3, y=y3))
        callback = CustomJS(args=dict(source=source), code="""
        var data = source.data;
        var n = valorn.value;
        x = {0}[:n]
        y = {1}[:n]
        source.change.emit();
        """.format(x3,y3))
        sliders[0] = Slider(start=1, end=nmax, value=1, step=1, title="Pasos", callback=callback, callback_policy='mouseup')
        p.line('x', 'y', source=source, color=tuple(np.random.randint(0, 256, 3)))
        callback.args["valorn"] = sliders[0]
    if isinstance(xiniciales, float):
        iteraciones(xiniciales)
    else:
        for xi in xiniciales:
            iteraciones(xi)
    show(column([widgetbox(sliders), p]))

In [138]:
def logística(x, r=4):
    return r * x*(1-x)

x = np.arange(0, 1, .01)

%time telaraña(logística, x, .6)

CPU times: user 321 ms, sys: 6.83 ms, total: 328 ms
Wall time: 322 ms


In [64]:
telaraña(lambda x:x/2, np.arange(-1,1,.01), [-1, 1])

In [43]:
a = np.arange(10)
np.repeat(a, 2)

ValueError: operands could not be broadcast together with shape (10,) (2,)